In [1]:
import ccxt.async_support as ccxt
import asyncio
import pandas as pd
import numpy as np
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler


In [2]:
symbol         = 'ALCH/USDT'
timeframe      = '1m'
risk_amount    = 10        #10$
leverage       = 5         #50$
dmi_period     = 14
atr_period     = 14
tp_multiplier  = 2.0
sl_multiplier  = 1.0
fee_rate       = 0.0002
max_positions  = 2
check_interval = 60     #seconds
model_threshold= 0.7    #70%

In [3]:
open_positions = []
model = GradientBoostingClassifier(n_estimators=100, random_state=42)
scaler = StandardScaler()

In [4]:
async def fetch_data(exchange, symbol:str, timeframe:str, limit:int=100)->pd.DataFrame:
    try:
        ohlcv = await exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        return df
    except Exception as e:
        logger.error(f"Error fetching data: {e}")
        return pd.DataFrame()

In [5]:
def calculate_indicators(df: pd.DataFrame)->pd.DataFrame:
    try:
        #df['high'].diff() = df['high'].shift(0) - df['high'].shift(1)
        #If the upward movement is stronger than the (next) downward movement, keep the upward movement.
        df['dm_plus']  = df['high'].diff().where(df['high'].diff() > df['low'].diff().shift(-1), 0)
        df['dm_minus'] = -df['low'].diff().where(-df['low'].diff() > df['high'].diff().shift(-1), 0)
        df['tr']       = pd.concat([df['high'] - df['low'],
                                   (df['high'] - df['close'].shift()).abs(),
                                   (df['low'] - df['close'].shift()).abs()], axis=1).max(axis=1)
        df['atr']      = df['tr'].rolling(window=atr_period).mean()
        df['di_plus']  = (df['dm_plus'].rolling(window=dmi_period).sum() / df['atr']) * 100
        df['di_minus'] = (df['dm_minus'].rolling(window=dmi_period).sum() / df['atr']) * 100

        ####
        df['price_change'] = df['close'].pct_change()
        df['di_diff']      = df['di_plus'] - df['di_minus']
        return df
    except Exception as e:
        logger.error(f"Error calculating indicators: {e}")
        return df

In [6]:
def train_model(df: pd.DataFrame)->None:
    try:
        features = ['price_change', 'di_diff', 'atr']
        X = df[features].dropna()
        y = (df['di_plus'].shift(-1) > df['di_minus'].shift(-1)).astype(int)[X.index]
        if len(X) < 10:
            return 
        X_scaled = scaler.fit_transform(X)
        model.fit(X_scaled, y)
    except Exception as e:
        logger.error(f"Error training model: {e}")

In [7]:
def get_dmi_signal(df: pd.DataFrame)->str:
    try:
        di_plus = df['di_plus'].iloc[-1]
        di_minus = df['di_minus'].iloc[-1]

SyntaxError: incomplete input (2211626832.py, line 4)

In [ ]:
####### Workign on it